In [1]:
import numpy as np
from Clasificador import Clasificador, ClasificadorRegresionLogistica, ClasificadorNaiveBayes
from Datos import Datos
from EstrategiaParticionado import ValidacionSimple, ValidacionCruzada

particionado = ValidacionCruzada(5)
cl = ClasificadorRegresionLogistica()

data = Datos('wdbc.data', predNominal=True)

# cl.entrenamiento(data.datos,data.nominalAtributos,data.diccionario)
# y = cl.clasifica(data.datos,data.nominalAtributos,data.diccionario)

# print(y)
val = cl.validacion(particionado,data)
print(val)

[0.05309735 0.03539823 0.05309735 0.01769912 0.00854701]
